In [ ]:
import numpy as np
import openpyxl
from scipy.spatial import distance

def load_latent_variables_from_excel(excel_path):
    """Excelファイルから潜在変数データとファイル名を読み込む関数"""
    latent_vectors = []
    file_names = []
    
    # Excelファイルを開く
    wb = openpyxl.load_workbook(excel_path)
    ws = wb.active

    # Excelの各行からファイル名と潜在変数ベクトルを読み込む
    for row in ws.iter_rows(min_row=1, values_only=True):
        file_name = row[0]  # 一列目がファイル名
        latent_vector = np.array(list(map(float, row[1].split(','))))  # 潜在変数をカンマ区切りで読み込む
        file_names.append(file_name)
        latent_vectors.append(latent_vector)

    return file_names, np.array(latent_vectors)

def find_top_k_similar(file_names, latent_vectors, new_latent, k=5):
    """既存の潜在変数と新たに生成した潜在変数とのユークリッド距離を計算し、上位k個を取得する"""
    distances = []
    
    # 既存の潜在変数と新しい潜在変数のユークリッド距離を計算
    for i, latent_vector in enumerate(latent_vectors):
        dist = np.linalg.norm(latent_vector - new_latent)  # ユークリッド距離を計算
        distances.append((i, dist))  # 各ベクトルのインデックスと距離を保存

    # 距離が小さい順にソート
    distances.sort(key=lambda x: x[1])

    # 上位k個を返す（ファイル名、距離）
    return [(file_names[idx], dist) for idx, dist in distances[:k]]

# Excelファイルから潜在変数をロード
excel_path = 'latent_variables_z.xlsx'
file_names, latent_vectors = load_latent_variables_from_excel(excel_path)

# 新しく生成した潜在変数 (例: バッチサイズが1で1つの潜在変数ベクトル)
new_latent_vector = np.random.rand(256)  # 256次元の潜在変数の例

# ユークリッド距離が最も小さい上位5つを取得
top_5_similar = find_top_k_similar(file_names, latent_vectors, new_latent_vector, k=5)

# 結果を表示
for file_name, dist in top_5_similar:
    print(f"File: {file_name}, Distance: {dist}")


In [ ]:
1. 次元削減（Dimensionality Reduction）
高次元空間での距離計算は計算量が大きくなりがちです。次元削減を行うことで、計算量を減らすことができます。代表的な方法には以下があります：

PCA（主成分分析）: 高次元データをより少ない次元に圧縮し、情報の損失を最小限に抑える方法。潜在変数の次元を削減して距離計算を行うことができます。
t-SNE や UMAP: データを2次元や3次元に投影し、距離関係を保持したまま次元を削減する手法。ただし、PCAに比べて計算コストが高い場合があります。
2. KD-Tree や Ball Tree を使った探索
空間内の近傍点を効率的に探索するためのデータ構造です。ユークリッド距離などの距離計算を効率化するために使われます。

KD-Tree: データの空間を軸に沿って再帰的に分割することで、最近傍探索を高速化します。ただし、高次元（100次元以上）では効率が落ちる場合があります。
Ball Tree: KD-Treeが高次元で効率が悪い場合に有効なデータ構造。空間を球体に分割し、距離計算を高速化します。
scikit-learn の NearestNeighbors クラスでこれらを簡単に実装できます：

python
コードをコピーする
from sklearn.neighbors import NearestNeighbors

# 潜在変数の行列（例: latent_vectors）に基づきKD-Treeを構築
nbrs = NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(latent_vectors)

# 新たに生成された潜在変数に対して最近傍探索を実行
distances, indices = nbrs.kneighbors([new_latent])
これにより、近傍点探索の計算量を効率化できます。

3. Approximate Nearest Neighbors (ANN)
厳密な最近傍探索ではなく、多少誤差が許される場合には、近似最近傍探索（ANN）を使うことができます。これにより、計算時間を大幅に短縮できます。

Faiss: Facebookが開発した、非常に効率的な近似最近傍検索ライブラリ。特にベクトルの探索に特化しており、GPUでの高速化も可能です。
python
コードをコピーする
import faiss

# 潜在変数の行列をFAISS用に準備
index = faiss.IndexFlatL2(latent_vectors.shape[1])  # L2距離（ユークリッド距離）を使用
index.add(latent_vectors)  # ベクトルをインデックスに追加

# 新しい潜在変数に対して最近傍探索を実行
distances, indices = index.search(new_latent.reshape(1, -1), k=5)
Faissは非常に高速な探索が可能で、特に大規模データに対して効果的です。

4. ブロック化やバッチ処理
距離計算をバッチ化することで、複数の潜在変数間の距離計算を並列処理やGPUを使って効率化できます。通常のループではなく、行列演算やバッチ処理を用いることで、計算時間を短縮できます。
例えば、NumPyやPyTorchを使ってユークリッド距離の計算をベクトル化することで、1回の演算で全ての距離を計算できます：
python
コードをコピーする
# 全ての潜在変数ベクトルとの距離を一気に計算
distances = np.linalg.norm(latent_vectors - new_latent, axis=1)


5. GPUの活用
PyTorchやTensorFlowなどのフレームワークを使って、距離計算をGPUで行うことも一つの手法です。特にベクトルの比較が大量に必要な場合、GPUの並列処理能力が有効です。
python
コードをコピーする
latent_vectors_tensor = torch.tensor(latent_vectors).to(device)  # GPUに転送
new_latent_tensor = torch.tensor(new_latent).to(device)
distances = torch.norm(latent_vectors_tensor - new_latent_tensor, dim=1)  # GPUで計算


6. ハッシュベースの検索
ハッシュテーブルを使って近傍点探索を高速化する方法もあります。

Locality-Sensitive Hashing (LSH): ベクトルの近さを反映したハッシュ関数を使い、計算量を削減するアルゴリズムです。特に大規模なデータセットで有効です。
